In [50]:
import pandas as pd
import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

from sklearn import preprocessing

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
df = pd.read_excel('sales_bronx_05.xls')

In [6]:
df = df[1:]

In [8]:
df.columns = df.iloc[0]

In [10]:
df = df[1:]

In [12]:
df.dtypes

2
BOROUGH                           object
NEIGHBORHOOD                      object
BUILDING CLASS CATEGORY           object
TAX CLASS AT PRESENT              object
BLOCK                             object
LOT                               object
EASE-MENT                         object
BUILDING CLASS AT PRESENT         object
ADDRESS                           object
APARTMENT NUMBER                  object
ZIP CODE                          object
RESIDENTIAL UNITS                 object
COMMERCIAL UNITS                  object
TOTAL UNITS                       object
LAND SQUARE FEET                  object
GROSS SQUARE FEET                 object
YEAR BUILT                        object
TAX CLASS AT TIME OF SALE         object
BUILDING CLASS AT TIME OF SALE    object
SALE PRICE                        object
SALE DATE                         object
dtype: object

In [14]:
df_cleaned = df [["NEIGHBORHOOD","BUILDING CLASS CATEGORY","ZIP CODE", "COMMERCIAL UNITS", "RESIDENTIAL UNITS", "GROSS SQUARE FEET",
         "SALE PRICE"]]

In [16]:
df_cleaned["BUILDING CLASS CATEGORY"]

3        01  ONE FAMILY HOMES                        
4        01  ONE FAMILY HOMES                        
5        01  ONE FAMILY HOMES                        
6        01  ONE FAMILY HOMES                        
7        01  ONE FAMILY HOMES                        
                             ...                     
11803    10  COOPS - ELEVATOR APARTMENTS             
11804    10  COOPS - ELEVATOR APARTMENTS             
11805    22  STORE BUILDINGS                         
11806    22  STORE BUILDINGS                         
11807    31  COMMERCIAL VACANT LAND                  
Name: BUILDING CLASS CATEGORY, Length: 11805, dtype: object

In [17]:
#finding dummies in all dataset
for col in df_cleaned.select_dtypes(include=[object]):
    print(df_cleaned[col].value_counts(dropna=False), "\n\n")

PARKCHESTER                  946
RIVERDALE                    881
BAYCHESTER                   796
MORRISANIA/LONGWOOD          796
WILLIAMSBRIDGE               747
SOUNDVIEW                    684
BRONXDALE                    567
THROGS NECK                  479
MORRIS PARK/VAN NEST         463
WAKEFIELD                    427
SCHUYLERVILLE/PELHAM BAY     392
BEDFORD PARK/NORWOOD         342
COUNTRY CLUB                 331
CASTLE HILL/UNIONPORT        330
MELROSE/CONCOURSE            329
KINGSBRIDGE/JEROME PARK      322
HIGHBRIDGE/MORRIS HEIGHTS    305
WESTCHESTER                  287
KINGSBRIDGE HTS/UNIV HTS     223
EAST TREMONT                 222
PELHAM PARKWAY NORTH         212
MOTT HAVEN/PORT MORRIS       210
PELHAM PARKWAY SOUTH         199
BATHGATE                     168
MOUNT HOPE/MOUNT EDEN        163
BELMONT                      148
FORDHAM                      138
HUNTS POINT                  133
CROTONA PARK                 128
CITY ISLAND                  124
WOODLAWN  

In [21]:
df_cleaned["ZIP CODE"] = df_cleaned["ZIP CODE"].astype(int)

/Users/diegoleoro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
df_cleaned["SALE PRICE"] = df_cleaned["SALE PRICE"].astype(float)

/Users/diegoleoro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_cleaned.dtypes

2
NEIGHBORHOOD                object
BUILDING CLASS CATEGORY     object
ZIP CODE                     int64
COMMERCIAL UNITS            object
RESIDENTIAL UNITS           object
GROSS SQUARE FEET           object
SALE PRICE                 float64
dtype: object

In [29]:
df_cleaned

2,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ZIP CODE,COMMERCIAL UNITS,RESIDENTIAL UNITS,GROSS SQUARE FEET,SALE PRICE
3,BATHGATE,01 ONE FAMILY HOMES,10457,1,0,3542,400000.0
4,BATHGATE,01 ONE FAMILY HOMES,10457,0,1,1474,0.0
5,BATHGATE,01 ONE FAMILY HOMES,10457,0,1,2070,0.0
6,BATHGATE,01 ONE FAMILY HOMES,10457,1,1,2040,0.0
7,BATHGATE,01 ONE FAMILY HOMES,10457,0,1,1374,0.0
...,...,...,...,...,...,...,...
11803,WOODLAWN,10 COOPS - ELEVATOR APARTMENTS,10470,0,0,0,85000.0
11804,WOODLAWN,10 COOPS - ELEVATOR APARTMENTS,10470,0,0,0,124077.0
11805,WOODLAWN,22 STORE BUILDINGS,10470,0,0,0,0.0
11806,WOODLAWN,22 STORE BUILDINGS,10470,2,0,13880,0.0


In [26]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

In [ ]:
df=encode_and_bind(df, 'dx')
df= encode_and_bind(df, 'dx_type')
df=encode_and_bind(df, 'sex')
df=encode_and_bind(df, 'localization')
df=encode_and_bind(df, 'cell_type')
df.head()

In [31]:
for f in ['NEIGHBORHOOD', 'BUILDING CLASS CATEGORY']:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_cleaned[f].values))
    df_cleaned[f] = lbl.transform(list(df_cleaned[f].values))

/Users/diegoleoro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [32]:
df_cleaned

2,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ZIP CODE,COMMERCIAL UNITS,RESIDENTIAL UNITS,GROSS SQUARE FEET,SALE PRICE
3,0,0,10457,1,0,3542,400000.0
4,0,0,10457,0,1,1474,0.0
5,0,0,10457,0,1,2070,0.0
6,0,0,10457,1,1,2040,0.0
7,0,0,10457,0,1,1374,0.0
...,...,...,...,...,...,...,...
11803,38,9,10470,0,0,0,85000.0
11804,38,9,10470,0,0,0,124077.0
11805,38,17,10470,0,0,0,0.0
11806,38,17,10470,2,0,13880,0.0


In [48]:
df_cleaned_ = df_cleaned [df_cleaned["SALE PRICE"]!=0]

In [49]:
df_cleaned_

2,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ZIP CODE,COMMERCIAL UNITS,RESIDENTIAL UNITS,GROSS SQUARE FEET,SALE PRICE
3,0,0,10457,1,0,3542,400000.0
8,0,0,10457,0,1,1526,320000.0
9,0,0,10457,0,1,1526,145000.0
10,0,0,10457,0,1,1496,283000.0
11,0,0,10457,0,1,1488,115000.0
...,...,...,...,...,...,...,...
11801,38,9,10470,0,0,0,69000.0
11802,38,9,10470,0,0,0,80000.0
11803,38,9,10470,0,0,0,85000.0
11804,38,9,10470,0,0,0,124077.0


In [51]:
df_cleaned_["GROSS SQUARE FEET"] = df_cleaned_["GROSS SQUARE FEET"].astype(int)

/Users/diegoleoro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
df_cleaned_[df_cleaned_["GROSS SQUARE FEET"]==0]

2,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ZIP CODE,COMMERCIAL UNITS,RESIDENTIAL UNITS,GROSS SQUARE FEET,SALE PRICE
24,0,0,10457,0,0,0,310000.0
47,0,1,10457,0,0,0,365000.0
57,0,1,10457,0,0,0,525000.0
85,0,4,10458,0,0,0,21622.0
86,0,4,10458,0,0,0,18574.0
...,...,...,...,...,...,...,...
11801,38,9,10470,0,0,0,69000.0
11802,38,9,10470,0,0,0,80000.0
11803,38,9,10470,0,0,0,85000.0
11804,38,9,10470,0,0,0,124077.0


In [ ]:
X = df_cleaned_[['NEIGHBORHOOD', 'BUILDING CLASS CATEGORY','ZIP CODE','COMMERCIAL UNITS', 'RESIDENTIAL UNITS','GROSS SQUARE FEET']]
y = df_cleaned_['SALE PRICE']